In [1]:
%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 120)

In [3]:
def clean_and_drop(df):
    # 只篩選有包含 '住' 用途的交易案
    df = df.loc[df['Main_Usage_Living'] == 1]
    df = df.drop(columns=['Main_Usage_Living'])
    
    # 因為都是 0
    df = df.drop(columns=['Non_City_Land_Usage', 'Main_Usage_Walk', 
                          'Main_Usage_Selling',
                          'Main_Usage_SnE'])
    
    # 只有 344 筆是包含工廠用途，且都不具住宅用途，故剔除
    df = df.loc[df['Main_Usage_Manufacturing'] == 0]
    df = df.drop(columns=['Main_Usage_Manufacturing'])
    
    # 只有 76 筆是包含停車用途，且都不具住宅用途，故剔除
    df = df.loc[df['Main_Usage_Parking'] == 0]
    df = df.drop(columns=['Main_Usage_Parking'])
    
    # 只有 78 筆有農業用途，且都不具住宅用途，故剔除
    df = df.loc[df['Main_Usage_Farm'] == 0]
    df = df.drop(columns=['Main_Usage_Farm'])
    
    # NOTICE: 我沒有錢，所以我先只買 6 房以下的
    df = df.loc[df['room'] < 6]
    
    df = df.loc[df['trading_floors_count'] == 1]
    
    # 雖然有 95 個樣本包含地下室，但是樣本太少，可能不足以推廣
    # 所以先剔除，剔除完後，都是 0 所以直接 drop
    df = df.loc[df['including_basement'] == 0]
    df = df.drop(columns=['including_basement'])
    
    # 所有的樣本都不包含人行道，所以直接去除這個 feature
    df = df.drop(columns=['including_arcade'])

    # 剔除交易樓層高度是 -1 (原本有一個樣本)
    df = df.loc[df['min_floors_height'] != -1]

    # 剔除交易建物是 0 個樓層的情況
    df = df.loc[df['building_total_floors'] != 0]
    
    # 因為車位交易 50 坪以上的資料只有 22 筆，所以先去除
    # 因為浮點數在硬體儲存會有小數點，故不能直接用 == 50.0 去比較
    df = df.loc[df['Parking_Area'] < 49.5]
    
    # 把農舍，廠辦踢掉
    df = df.loc[df['Building_Types'] < 8]

    # 把超大轉移坪數刪掉
    df = df.loc[df['Transfer_Total_Ping'] < 150]
    
    # 我先刪除 area_m2, 因為覺得跟 area_ping 的意義很類似，但是不確定會不會有些微差距。
    # 因為在 future data 中，manager 都是 0，所以也把這個欄位刪除
    # trading_floor_count 有 0 的情況，這樣應該不是房屋交易
    df = df.drop(columns=['address', 'area_m2', 'manager', 'Building_Material_stone', 
                     'TDATE', 'Total_price', '編號'])
    
    # Convert the categorical features' dtype to 'category'
    category_columns = ['Type', 'Month', 'Month_raw',
                       'City_Land_Usage', 'Main_Usage_Business',
                       'Building_Material_S', 'Building_Material_R', 'Building_Material_C',
                       'Building_Material_steel', 'Building_Material_B', 
                       'Building_Material_W', 'Building_Material_iron',
                       'Building_Material_tile', 'Building_Material_clay',
                       'Building_Material_RC_reinforce',
                       'Parking_Space_Types', 'Building_Types']
    df.loc[:, category_columns] = df.loc[:, category_columns].astype('category')
    return df

In [5]:
def split_features_target(df):
    X = df.drop(columns=['Unit_Price_Ping'])
    y = df['Unit_Price_Ping']
    return X, y

def train(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

def eval(model, X_test, y_test):
    from sklearn.metrics import (r2_score, 
                                 mean_absolute_error, 
                                 mean_squared_error)
    
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print(f'R2 score: {r2}')
    print(f'MAE score: {mae}')
    print(f'MSE score: {mse}')
    return r2, mae, mse

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
df_future = pd.read_csv('../temp_future/output_feature/clean_data_future_train.csv')
df_future_test = pd.read_csv('../temp_future/output_feature/clean_data_future_test.csv')

In [8]:
df_future = clean_and_drop(df_future)
df_future = df_future.sample(frac=1, random_state=0)
X_train, y_train = split_features_target(df_future)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [9]:
df_future_test = clean_and_drop(df_future_test)
X_test, y_test = split_features_target(df_future_test)

In [ ]:
df_future['Month'].unique()

In [ ]:
X_val.shape

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = train(DecisionTreeRegressor(max_depth=20), X_train, y_train)

print('Training performance: ')
eval(model, X_train, y_train)
print()
print('Evaluation performance: ')
eval(model, X_val, y_val)
print()
print('Test performance:')
eval(model, X_test, y_test)

In [10]:
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm


avg_train_r2, avg_train_mae, avg_train_mse = 0, 0, 0
avg_val_r2, avg_val_mae, avg_val_mse = 0, 0, 0
avg_test_r2, avg_test_mae, avg_test_mse = 0, 0, 0

for i in tqdm(range(100)):
    model = train(DecisionTreeRegressor(max_depth=20), X_train, y_train)
    r2, mae, mse = eval(model, X_train, y_train)
    avg_train_r2 += r2
    avg_train_mae += mae
    avg_train_mse += mse
    
    r2, mae, mse = eval(model, X_val, y_val)
    avg_val_r2 += r2
    avg_val_mae += mae
    avg_val_mse += mse
    
    r2, mas, mse = eval(model, X_test, y_test)
    avg_test_r2 += r2
    avg_test_mae += mae
    avg_test_mse += mse

avg_train_r2 /= 100.
avg_train_mae /= 100.
avg_train_mse /= 100.

avg_val_r2 /= 100.
avg_val_mae /= 100.
avg_val_mse /= 100.

avg_test_r2 /= 100.
avg_test_mae /= 100.
avg_test_mse /= 100.

print('Training Performance:')
print(f'R2 score: {avg_train_r2}')
print(f'MAE score: {avg_train_mae}')
print(f'MSE score: {avg_train_mse}')
print()

print('Validation Performance:')
print(f'R2 score: {avg_val_r2}')
print(f'MAE score: {avg_val_mae}')
print(f'MSE score: {avg_val_mse}')
print()

print('Testing Performance')
print(f'R2 score: {avg_test_r2}')
print(f'MAE score: {avg_test_mae}')
print(f'MSE score: {avg_test_mse}')

  1%|█▏                                                                                                                         | 1/100 [00:01<02:33,  1.55s/it]

R2 score: 0.9929424971568468
MAE score: 7948.401799083027
MSE score: 220200924.47428194
R2 score: 0.9703920226017639
MAE score: 14635.799883304284
MSE score: 881010787.3632598
R2 score: 0.8737606676089242
MAE score: 40943.83951898885
MSE score: 4690826894.327063


  2%|██▍                                                                                                                        | 2/100 [00:03<02:31,  1.55s/it]

R2 score: 0.9929424971568468
MAE score: 7948.401799083027
MSE score: 220200924.474282
R2 score: 0.96897422651496
MAE score: 14718.389977484296
MSE score: 923198527.1725355
R2 score: 0.8464227187785157
MAE score: 43351.90276547711
MSE score: 5706655980.084196


  3%|███▋                                                                                                                       | 3/100 [00:04<02:30,  1.55s/it]

...

R2 score: 0.9929424971568468
MAE score: 7948.401799083029
MSE score: 220200924.47428194
R2 score: 0.970912674994769
MAE score: 14568.023631923712
MSE score: 865518328.4041692
R2 score: 0.8789080048162916
MAE score: 40462.742339247256
MSE score: 4499561087.156206


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 99/100 [02:31<00:01,  1.53s/it]

R2 score: 0.9929424971568468
MAE score: 7948.401799083025
MSE score: 220200924.47428194
R2 score: 0.9698142236541545
MAE score: 14650.387791768953
MSE score: 898203691.1176865
R2 score: 0.8437271308291409
MAE score: 44924.285490430084
MSE score: 5806819187.615902


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:33<00:00,  1.53s/it]

R2 score: 0.9929424971568468
MAE score: 7948.401799083027
MSE score: 220200924.474282
R2 score: 0.9695215410375694
MAE score: 14640.337490070386
MSE score: 906912713.6563395
R2 score: 0.8519250269226879
MAE score: 43100.67658908128
MSE score: 5502200090.34929
Training Performance:
R2 score: 0.9929424971568467
MAE score: 7948.401799083019
MSE score: 220200924.4742816

Validation Performance:
R2 score: 0.9693143335274214
MAE score: 14669.77031213327
MSE score: 913078351.0184444

Testing Performance
R2 score: 0.8580127665256704
MAE score: 14669.77031213327
MSE score: 5275990618.907655


In [ ]:
# from sklearn.svm import LinearSVR

# model = LinearSVR(C=1.0, epsilon=0.0, verbose=True, max_iter=300, random_state=1207)
# train(model, X_train, y_train)

# print('Training performance: ')
# eval(model, X_train, y_train)
# print()
# print('Evaluation performance: ')
# eval(model, X_test, y_test)

In [ ]:
# Too slow
# The fit time complexity is more than quadratic
# with the number of samples which makes it hard
# to scale to datasets with more than a couple of
# 10000 samples. For large datasets consider using
# LinearSVR or SGDRegressor instead, possibly
# after a Nystroem transformer.
from sklearn.svm import SVR

model = SVR(C=1.0, epsilon=0.0, verbose=True)
train(model, X_train, y_train)

print('Training performance: ')
eval(model, X_train, y_train)
print()
print('Evaluation performance: ')
eval(model, X_test, y_test)